In [1]:
import pandas as pd
import numpy as np
import os
import glob
from nilearn import datasets, image
from nilearn import plotting  

In [2]:
path = '/media/hcs-psy-narun/Alina_results/Alina3/mainset_outputs/enet_coeffs_tables/'
wway = '/media/DataD800/Alina/atlases/'

In [3]:
#load table of matchings between Glasser parcels and Cole-Anticevic nets
match_net_tab = pd.read_csv(wway+'Glasser-Cole_match_table.txt')
display(match_net_tab)

,orig_order,left_first_order,Cole_net_order,name_Glasser,Cole_net_name
0,1,181,1,R_V1_ROI,Visual1
1,2,182,2,R_MST_ROI,Visual2
2,3,183,2,R_V6_ROI,Visual2
3,4,184,2,R_V2_ROI,Visual2
4,5,185,2,R_V3_ROI,Visual2
...,...,...,...,...,...
355,356,176,9,L_STSva_ROI,Default
356,357,177,9,L_TE1m_ROI,Default
357,358,178,4,L_PI_ROI,Cingulo-Opercular
358,359,179,4,L_a32pr_ROI,Cingulo-Opercular


In [4]:
#load table with voxels coordinate for Glasser atlas in voxspace

gla_mni = pd.read_csv(wway+'HCP-MMP1_UniqueRegionList.txt')
gla_mni.iloc[:,-2] = gla_mni.iloc[:,-2].round().astype('Int64', errors='ignore')
gla_mni.iloc[:,-3] = gla_mni.iloc[:,-3].round().astype('Int64', errors='ignore')
gla_mni.iloc[:,-4] = gla_mni.iloc[:,-4].round().astype('Int64', errors='ignore')
display(gla_mni)

,regionName,regionLongName,regionIdLabel,LR,region,Lobe,cortex,regionID,Cortex_ID,x-cog,y-cog,z-cog,volmm
0,V1_L,Primary_Visual_Cortex_L,1_L,L,V1,Occ,Primary_Visual,1,1,100,41,72,6717
1,MST_L,Medial_Superior_Temporal_Area_L,2_L,L,MST,Occ,MT+_Complex_and_Neighboring_Visual_Areas,2,5,132,59,82,336
2,V6_L,Sixth_Visual_Area_L,3_L,L,V6,Occ,Dorsal_Stream_Visual,3,3,105,44,104,1009
3,V2_L,Second_Visual_Area_L,4_L,L,V2,Occ,Early_Visual,4,2,102,44,74,6220
4,V3_L,Third_Visual_Area_L,5_L,L,V3,Occ,Early_Visual,5,2,108,41,77,4994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,STSva_R,Area_STSv_anterior_R,376_R,R,STSva,Temp,Auditory_Association,376,11,33,118,54,1720
356,TE1m_R,Area_TE1_Middle_R,377_R,R,TE1m,Temp,Lateral_Temporal,377,14,26,102,53,2592
357,PI_R,Para-Insular_Area_R,378_R,R,PI,Temp,Insular_and_Frontal_Opercular,378,12,47,123,58,898
358,a32pr_R,Area_anterior_32_prime_R,379_R,R,a32pr,Fr,Anterior_Cingulate_and_Medial_Prefrontal,379,19,82,154,103,827


In [5]:
#load fsl template for transformation

niimg = image.load_img(wway+'MNI152_T1_1mm_brain.nii.gz') #can be found at /usr/local/fsl/data/standard/

In [6]:
# transformation from voxel to MNI coordinates

dmn_coords = [tuple(list(gla_mni.iloc[i,-4:-1])) for i in range(len(gla_mni.index))]
dmn_coords_MNI = [image.coord_transform(dmn_coords[i][0], dmn_coords[i][1],dmn_coords[i][2], niimg.affine) for i in range(len(dmn_coords))]
#print(dmn_coords_MNI)
view = plotting.view_markers(dmn_coords_MNI, marker_size=10)
view.open_in_browser() 
gla_mni.iloc[:,-4] = [dmn_coords_MNI[i][0] for i in  range(len(dmn_coords_MNI))]
gla_mni.iloc[:,-3] = [dmn_coords_MNI[i][1] for i in  range(len(dmn_coords_MNI))]
gla_mni.iloc[:,-2] = [dmn_coords_MNI[i][2] for i in  range(len(dmn_coords_MNI))]
display(gla_mni)

,regionName,regionLongName,regionIdLabel,LR,region,Lobe,cortex,regionID,Cortex_ID,x-cog,y-cog,z-cog,volmm
0,V1_L,Primary_Visual_Cortex_L,1_L,L,V1,Occ,Primary_Visual,1,1,-10.0,-85.0,0.0,6717
1,MST_L,Medial_Superior_Temporal_Area_L,2_L,L,MST,Occ,MT+_Complex_and_Neighboring_Visual_Areas,2,5,-42.0,-67.0,10.0,336
2,V6_L,Sixth_Visual_Area_L,3_L,L,V6,Occ,Dorsal_Stream_Visual,3,3,-15.0,-82.0,32.0,1009
3,V2_L,Second_Visual_Area_L,4_L,L,V2,Occ,Early_Visual,4,2,-12.0,-82.0,2.0,6220
4,V3_L,Third_Visual_Area_L,5_L,L,V3,Occ,Early_Visual,5,2,-18.0,-85.0,5.0,4994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,STSva_R,Area_STSv_anterior_R,376_R,R,STSva,Temp,Auditory_Association,376,11,57.0,-8.0,-18.0,1720
356,TE1m_R,Area_TE1_Middle_R,377_R,R,TE1m,Temp,Lateral_Temporal,377,14,64.0,-24.0,-19.0,2592
357,PI_R,Para-Insular_Area_R,378_R,R,PI,Temp,Insular_and_Frontal_Opercular,378,12,43.0,-3.0,-14.0,898
358,a32pr_R,Area_anterior_32_prime_R,379_R,R,a32pr,Fr,Anterior_Cingulate_and_Medial_Prefrontal,379,19,8.0,28.0,31.0,827


In [7]:
#Read modality names

mods = sorted([n.split('/')[-1].split('_')[0] for n in sorted(list(glob.glob(path+'*coeffs*')))])
#mods = [mods[0], mods[1], mods[11], mods[10], mods[8], mods[2], mods[3], mods[4], mods[6], mods[7], mods[9], mods[12]]
mods.remove('all')
mods.remove('non-task')
mods.remove('rest-mean')
mods.remove('rest-pca75')
mods.remove('task')
mods.remove('top-task')
print(mods)


['VolBrain', 'cort', 'emo', 'gam', 'lan', 'mean', 'mean-TASKS', 'mot', 'rel', 'rest', 'rest', 'soc', 'subc', 'surf', 'wm']


In [8]:
#load modalities tables into dictionaries and calculate mean and std for each parcel

dct={}
dct_std={}
for mod in mods:
    dct[mod] = pd.read_csv(glob.glob(path+mod+'*coeffs*')[0], index_col=0).mean()
    dct_std[mod] = pd.read_csv(glob.glob(path+mod+'*coeffs*')[0], index_col=0).std()
    


In [9]:
glob.glob(path+mods[0]+'*coeffs*')[0]

'/media/hcs-psy-narun/Alina_results/Alina3/mainset_outputs/enet_coeffs_tables/VolBrain_enet_coeffs_folds.csv'

In [41]:
dct['wm']

R_V1_ROI          0.059033
R_MST_ROI         0.013740
R_V6_ROI         -0.025129
R_V2_ROI          0.016610
R_V3_ROI         -0.036456
                    ...   
PALLIDUM_RIGHT   -0.128213
PUTAMEN_LEFT      0.031274
PUTAMEN_RIGHT     0.077944
THALAMUS_LEFT     0.050988
THALAMUS_RIGHT    0.115373
Length: 379, dtype: float64

In [10]:
dct_std['wm']

R_V1_ROI          0.058297
R_MST_ROI         0.056462
R_V6_ROI          0.055785
R_V2_ROI          0.029735
R_V3_ROI          0.050221
                    ...   
PALLIDUM_RIGHT    0.118751
PUTAMEN_LEFT      0.069386
PUTAMEN_RIGHT     0.116742
THALAMUS_LEFT     0.080118
THALAMUS_RIGHT    0.067163
Length: 379, dtype: float64

In [11]:
# replace short parcels name in gla_mni table because of some name mistakes in previos table
mp=[n.split('_')[0]+'_'+n.split('_')[1] for n in np.concatenate([list(dct['emo'].index[:-19])[180:], list(dct['emo'].index[:-19])[:180]], axis=0, dtype=str)]
gla_mni['regionName'] = mp
display(gla_mni)
gla_mni.to_csv(path+'HCP-MMP1_UniqueRegionList_MNI_transformed.csv')

,regionName,regionLongName,regionIdLabel,LR,region,Lobe,cortex,regionID,Cortex_ID,x-cog,y-cog,z-cog,volmm
0,L_V1,Primary_Visual_Cortex_L,1_L,L,V1,Occ,Primary_Visual,1,1,-10.0,-85.0,0.0,6717
1,L_MST,Medial_Superior_Temporal_Area_L,2_L,L,MST,Occ,MT+_Complex_and_Neighboring_Visual_Areas,2,5,-42.0,-67.0,10.0,336
2,L_V6,Sixth_Visual_Area_L,3_L,L,V6,Occ,Dorsal_Stream_Visual,3,3,-15.0,-82.0,32.0,1009
3,L_V2,Second_Visual_Area_L,4_L,L,V2,Occ,Early_Visual,4,2,-12.0,-82.0,2.0,6220
4,L_V3,Third_Visual_Area_L,5_L,L,V3,Occ,Early_Visual,5,2,-18.0,-85.0,5.0,4994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,R_STSva,Area_STSv_anterior_R,376_R,R,STSva,Temp,Auditory_Association,376,11,57.0,-8.0,-18.0,1720
356,R_TE1m,Area_TE1_Middle_R,377_R,R,TE1m,Temp,Lateral_Temporal,377,14,64.0,-24.0,-19.0,2592
357,R_PI,Para-Insular_Area_R,378_R,R,PI,Temp,Insular_and_Frontal_Opercular,378,12,43.0,-3.0,-14.0,898
358,R_a32pr,Area_anterior_32_prime_R,379_R,R,a32pr,Fr,Anterior_Cingulate_and_Medial_Prefrontal,379,19,8.0,28.0,31.0,827


In [12]:
#Assemble tabled for all structural modalities

for mod in ['VolBrain', 'cort', 'surf', 'subc']:#mods[:4]:
    print(mod)
    tt1 = dct[mod].abs().sort_values(ascending=False)
    tt2 = dct[mod].reindex(index = tt1.index)
    tt3 = dct_std[mod].reindex(index = tt1.index)
    tt4 =  pd.DataFrame({'Mean Coeff': tt2,
                      'σ Coeff':tt3})
    
    tab1 = dct[mod].abs().sort_values(ascending=False)[:20]
    tab2 = dct[mod].reindex(index = tab1.index)
    tab3 = dct_std[mod].reindex(index = tab1.index)
    dft = pd.DataFrame({'Mean Coeff': tab2,
                      'σ Coeff':tab3})
    display(dft)
    dft.to_csv(path+str(mod)+'_top20_feature_coef.csv')
    tt4.to_csv(path+str(mod)+'_all_feature_coef.csv')
    print(' ')


VolBrain


,Mean Coeff,σ Coeff
FS_IntraCranial_Vol,2.283078,1.931946
FS_TotCort_GM_Vol,1.589493,0.956885
FS_Tot_WM_Vol,-0.692513,1.044007
FS_SubCort_GM_Vol,-0.667749,0.744858
FS_BrainSegVol_eTIV_Ratio,0.466914,1.071568


 
cort


,Mean Coeff,σ Coeff
rh_G_and_S_transv_frontopol,-0.314739,0.337680
lh_G_parietal_sup,-0.300250,0.288610
rh_G_cuneus,0.239435,0.310293
lh_G_temp_sup-G_T_transv,0.212853,0.383228
lh_G_front_middle,-0.152267,0.120862
lh_S_cingul-Marginalis,-0.124441,0.109883
rh_G_front_middle,-0.121146,0.094692
lh_S_circular_insula_ant,0.104514,0.080895
lh_G_occipital_sup,-0.104223,0.117848
rh_G_rectus,0.096981,0.080484


 
surf


,Mean Coeff,σ Coeff
lh_G_temp_sup-Plan_polar,0.784228,0.590238
lh_G_rectus,-0.643758,0.350450
rh_Pole_temporal,0.632934,0.447862
lh_S_central,0.358077,0.229661
rh_G_precentral,0.341925,0.194333
lh_G_occipital_sup,0.291951,0.282142
lh_S_calcarine,0.287215,0.243968
lh_G_cuneus,0.243617,0.275905
rh_G_temp_sup-Plan_polar,0.232256,0.222161
lh_G_temporal_inf,0.225879,0.195863


 
subc


,Mean Coeff,σ Coeff
Left-Cerebellum-Cortex,1.373743,0.894341
Left-Hippocampus,0.677279,0.320640
Brain-Stem,-0.542168,0.418476
Left-Thalamus-Proper,0.382800,0.369082
Left-Pallidum,-0.291845,0.241172
Right-Hippocampus,0.276190,0.301483
Left-Caudate,0.265720,0.201353
Right-Caudate,0.202912,0.192616
Right-Cerebellum-Cortex,0.202892,0.224841
Left-Putamen,-0.181213,0.161422


In [23]:
#create tables for task modalities
#top20
for mod in ['emo', 'lan', 'mot', 'rel', 'soc', 'wm']:
    if mod != 'gam':
        print(mod)
        tab1 = dct[mod].abs().sort_values(ascending=False)[:20]
        tab2 = dct[mod].reindex(index = tab1.index)
        tab3 = dct_std[mod].reindex(index = tab1.index)
        
        tabnet = match_net_tab[match_net_tab['name_Glasser'].isin(list(tab1.index))].iloc[:,3:]
        tabnet = tabnet.set_index(tabnet['name_Glasser'])
        tabnet = tabnet.reindex(index=tab1.index)
        tabnet['coeff'] = np.array(tab2)
        tabnet['σ Coeff'] = np.array(tab3)
        tabnet = tabnet.reset_index()
        tabnet = tabnet.drop('name_Glasser', axis=1)
        tabnet = tabnet.rename(columns={'index': 'name_Glasser'})
        tabnet['Cole_net_name'] = tabnet['Cole_net_name'].fillna('subcortex')
        tabnet['freq'] = [tabnet['Cole_net_name'].value_counts()[val] for val in tabnet['Cole_net_name']]
        tabnet = tabnet.sort_values(by=['freq', 'Cole_net_name', 'coeff'], ascending=False)
        tabnet = tabnet.drop('freq', axis=1)
        tabnet = tabnet.reset_index(drop=True)
        
        init_lbl_gla = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]
        #print(init_lbl_gla)
        tabnet['name_Glasser'] = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]
        tabnet = tabnet.set_index(tabnet['name_Glasser'])
        gla = gla_mni[gla_mni['regionName'].isin(tabnet['name_Glasser'])].loc[:,('regionName', 'regionLongName', 'Lobe', 'cortex', 'x-cog', 'y-cog', 'z-cog')]
        gla = gla.set_index(gla['regionName'])
        gla = gla.reindex(index = tabnet.index)

        tabnet = pd.concat([tabnet, gla], axis=1)

        col_nm = list(tabnet.columns)
        col_nm = [col_nm[i] for i in [0,7, 1, 8,9,10, 2, 3]]

        tabnet =tabnet.loc[:,col_nm]
        tabnet = tabnet.reset_index(drop=True)        
        #tabnet = tabnet.fillna(0)
        tabnet['name_Glasser']=init_lbl_gla
        nnms = ['Glasser ROI', 'Brain Region', 'Network', 'x-MNI', 'y-MNI', 'z-MNI', 'Mean Coeff', 'σ Coeff']
        tabnet.columns = nnms
        tabnet = tabnet.round(2)
        tabnet['x-MNI'] = tabnet['x-MNI'].round().astype('Int64')
        tabnet['y-MNI'] = tabnet['y-MNI'].round().astype('Int64')
        tabnet['z-MNI'] = tabnet['z-MNI'].round().astype('Int64')
        tabnet.index = tabnet['Glasser ROI']
        tabnet.pop('Glasser ROI')
       
        
        display(tabnet)
        tabnet.to_csv(path+str(mod)+'_top20_feature_coef.csv')
        ##tabnet.to_csv(path+str(mod)+'_all_feature_coef.csv')
        print(' ')

emo


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_8BL,Dorsolateral_Prefrontal,Default,11,43,48,0.34,0.62
R_PreS,Medial_Temporal,Default,20,-31,-10,0.22,0.55
L_PreS,Medial_Temporal,Default,-20,-31,-11,0.22,0.24
R_47m,Orbital_and_Polar_Frontal,Default,32,31,-18,0.18,0.30
L_p32pr,Anterior_Cingulate_and_Medial_Prefrontal,Cingulo-Opercular,-9,15,38,-0.15,0.10
R_p24,Anterior_Cingulate_and_Medial_Prefrontal,Cingulo-Opercular,4,35,17,-0.21,0.37
L_23c,Paracentral_Lobular_and_Mid_Cingulate,Cingulo-Opercular,-13,-30,42,-0.25,0.42
R_p24pr,Anterior_Cingulate_and_Medial_Prefrontal,Cingulo-Opercular,4,-4,41,-0.27,0.35
R_V6,Dorsal_Stream_Visual,Visual2,18,-77,31,0.22,0.22


 
lan


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
L_8BM,Anterior_Cingulate_and_Medial_Prefrontal,Frontoparietal,-6,33,44,0.69,0.44
L_AVI,Insular_and_Frontal_Opercular,Frontoparietal,-31,25,-4,0.43,0.37
L_a47r,Inferior_Frontal,Frontoparietal,-41,48,-13,0.30,0.32
R_31a,Posterior_Cingulate,Frontoparietal,6,-40,43,-0.30,0.28
R_p10p,Orbital_and_Polar_Frontal,Frontoparietal,23,61,1,-0.30,0.22
L_IFSa,Inferior_Frontal,Frontoparietal,-47,33,9,-0.58,0.49
L_9m,Anterior_Cingulate_and_Medial_Prefrontal,Default,-7,54,22,0.54,0.53
L_47l,Inferior_Frontal,Default,-47,29,-12,0.48,0.38
R_TGd,Lateral_Temporal,Default,35,14,-37,0.41,0.38


 
mot


...

,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_47m,Orbital_and_Polar_Frontal,Default,32,31,-18,-0.15,0.10
R_POS1,Posterior_Cingulate,Default,11,-57,15,-0.16,0.07
R_47l,Inferior_Frontal,Default,44,32,-15,-0.17,0.11
L_47m,Orbital_and_Polar_Frontal,Default,-37,31,-17,-0.20,0.13
L_FST,MT+_Complex_and_Neighboring_Visual_Areas,Visual2,-48,-68,5,0.19,0.12
R_VIP,Superior_Parietal,Visual2,21,-63,64,0.18,0.11
L_PIT,Ventral_Stream_Visual,Visual2,-47,-77,-11,-0.16,0.08
L_55b,Premotor,Language,-49,-1,50,0.17,0.09
L_SFL,Dorsolateral_Prefrontal,Language,-8,17,63,0.15,0.06


 
soc


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_STSda,Auditory_Association,Default,51,-1,-17,0.74,0.65
R_STSva,Auditory_Association,Default,57,-8,-18,0.28,0.23
R_7m,Posterior_Cingulate,Default,5,-60,33,0.26,0.17
R_PGi,Inferior_Parietal,Default,52,-60,30,0.11,0.08
L_PHA2,Medial_Temporal,Default,-31,-36,-14,0.11,0.23
L_IPS1,Dorsal_Stream_Visual,Visual2,-24,-74,38,-0.08,0.11
L_V3CD,MT+_Complex_and_Neighboring_Visual_Areas,Visual2,-35,-90,9,-0.10,0.13
R_LO1,MT+_Complex_and_Neighboring_Visual_Areas,Visual2,43,-81,5,-0.12,0.20
L_V3B,Dorsal_Stream_Visual,Visual2,-28,-83,16,-0.40,0.48


 
wm


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_33pr,Anterior_Cingulate_and_Medial_Prefrontal,Frontoparietal,3,11,28,0.34,0.16
R_7Pm,Superior_Parietal,Frontoparietal,5,-67,50,0.32,0.12
R_d32,Anterior_Cingulate_and_Medial_Prefrontal,Frontoparietal,6,39,27,0.29,0.24
L_7Pm,Superior_Parietal,Frontoparietal,-5,-68,49,0.27,0.09
R_AVI,Insular_and_Frontal_Opercular,Frontoparietal,32,25,-4,0.26,0.09
R_EC,Medial_Temporal,Default,20,-11,-27,0.31,0.06
L_TE1a,Lateral_Temporal,Default,-60,-4,-25,-0.24,0.08
L_31pv,Posterior_Cingulate,Default,-10,-44,33,-0.24,0.09
R_PHA1,Medial_Temporal,Default,20,-34,-17,-0.30,0.12


In [24]:
#create tables for task modalities
#all parcels
for mod in['emo', 'lan', 'mot', 'rel', 'soc', 'wm']:
    if mod != 'gam':
        print(mod)
        tab1 = dct[mod].abs().sort_values(ascending=False)#[:20]
        tab2 = dct[mod].reindex(index = tab1.index)
        tab3 = dct_std[mod].reindex(index = tab1.index)
        
        tabnet = match_net_tab[match_net_tab['name_Glasser'].isin(list(tab1.index))].iloc[:,3:]
        tabnet = tabnet.set_index(tabnet['name_Glasser'])
        tabnet = tabnet.reindex(index=tab1.index)
        tabnet['coeff'] = np.array(tab2)
        tabnet['σ Coeff'] = np.array(tab3)
        tabnet = tabnet.reset_index()
        tabnet = tabnet.drop('name_Glasser', axis=1)
        tabnet = tabnet.rename(columns={'index': 'name_Glasser'})
        tabnet['Cole_net_name'] = tabnet['Cole_net_name'].fillna('subcortex')
        tabnet['freq'] = [tabnet['Cole_net_name'].value_counts()[val] for val in tabnet['Cole_net_name']]
        tabnet = tabnet.sort_values(by=['freq', 'Cole_net_name', 'coeff'], ascending=False)
        tabnet = tabnet.drop('freq', axis=1)
        tabnet = tabnet.reset_index(drop=True)
        
        init_lbl_gla = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]
        #print(init_lbl_gla)
        tabnet['name_Glasser'] = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]
        tabnet = tabnet.set_index(tabnet['name_Glasser'])
        gla = gla_mni[gla_mni['regionName'].isin(tabnet['name_Glasser'])].loc[:,('regionName', 'regionLongName', 'Lobe', 'cortex', 'x-cog', 'y-cog', 'z-cog')]
        gla = gla.set_index(gla['regionName'])
        gla = gla.reindex(index = tabnet.index)

        tabnet = pd.concat([tabnet, gla], axis=1)

        col_nm = list(tabnet.columns)
        col_nm = [col_nm[i] for i in [0,7, 1, 8,9,10, 2, 3]]

        tabnet =tabnet.loc[:,col_nm]
        tabnet = tabnet.reset_index(drop=True)        
        #tabnet = tabnet.fillna(0)
        tabnet['name_Glasser']=init_lbl_gla
        nnms = ['Glasser ROI', 'Brain Region', 'Network', 'x-MNI', 'y-MNI', 'z-MNI', 'Mean Coeff', 'σ Coeff']
        tabnet.columns = nnms
        tabnet = tabnet.round(2)
        tabnet['x-MNI'] = tabnet['x-MNI'].round().astype('Int64')
        tabnet['y-MNI'] = tabnet['y-MNI'].round().astype('Int64')
        tabnet['z-MNI'] = tabnet['z-MNI'].round().astype('Int64')
        tabnet.index = tabnet['Glasser ROI']
        tabnet.pop('Glasser ROI')
       
        
        display(tabnet)
        #tabnet.to_csv(path+str(mod)+'_top20_feature_coef.csv')
        tabnet.to_csv(path+str(mod)+'_all_feature_coef.csv')
        print(' ')

emo


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_8BL,Dorsolateral_Prefrontal,Default,11,43,48,0.34,0.62
R_PreS,Medial_Temporal,Default,20,-31,-10,0.22,0.55
L_PreS,Medial_Temporal,Default,-20,-31,-11,0.22,0.24
R_47m,Orbital_and_Polar_Frontal,Default,32,31,-18,0.18,0.30
L_d23ab,Posterior_Cingulate,Default,-4,-42,31,0.13,0.06
...,...,...,...,...,...,...,...
R_AAIC,Insular_and_Frontal_Opercular,Orbito-Affective,35,15,-12,-0.00,0.02
R_TF,Medial_Temporal,Ventral_Multimodal,40,-20,-31,0.02,0.04
L_PeEc,Medial_Temporal,Ventral_Multimodal,-30,-9,-34,0.01,0.03


 
lan


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
L_9m,Anterior_Cingulate_and_Medial_Prefrontal,Default,-7,54,22,0.54,0.53
L_47l,Inferior_Frontal,Default,-47,29,-12,0.48,0.38
R_TGd,Lateral_Temporal,Default,35,14,-37,0.41,0.38
L_PGi,Inferior_Parietal,Default,-49,-65,27,0.36,0.28
L_9p,Dorsolateral_Prefrontal,Default,-19,47,38,0.28,0.24
...,...,...,...,...,...,...,...
L_Pir,Insular_and_Frontal_Opercular,Orbito-Affective,-33,5,-18,-0.04,0.07
L_TF,Medial_Temporal,Ventral_Multimodal,-42,-21,-31,0.07,0.12
R_PeEc,Medial_Temporal,Ventral_Multimodal,27,-9,-35,-0.01,0.05


 
mot


...

,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_7m,Posterior_Cingulate,Default,5,-60,33,0.11,0.07
L_TE1m,Lateral_Temporal,Default,-66,-25,-17,0.09,0.06
L_7m,Posterior_Cingulate,Default,-6,-62,34,0.08,0.05
R_TGd,Lateral_Temporal,Default,35,14,-37,0.07,0.05
L_s32,Anterior_Cingulate_and_Medial_Prefrontal,Default,-7,33,-18,0.05,0.04
...,...,...,...,...,...,...,...
R_Pir,Insular_and_Frontal_Opercular,Orbito-Affective,32,7,-19,-0.04,0.09
L_PeEc,Medial_Temporal,Ventral_Multimodal,-30,-9,-34,0.06,0.05
R_PeEc,Medial_Temporal,Ventral_Multimodal,27,-9,-35,0.01,0.03


 
soc


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_STSda,Auditory_Association,Default,51,-1,-17,0.74,0.65
R_STSva,Auditory_Association,Default,57,-8,-18,0.28,0.23
R_7m,Posterior_Cingulate,Default,5,-60,33,0.26,0.17
R_PGi,Inferior_Parietal,Default,52,-60,30,0.11,0.08
L_PHA2,Medial_Temporal,Default,-31,-36,-14,0.11,0.23
...,...,...,...,...,...,...,...
R_AAIC,Insular_and_Frontal_Opercular,Orbito-Affective,35,15,-12,-0.01,0.03
L_PeEc,Medial_Temporal,Ventral_Multimodal,-30,-9,-34,0.06,0.12
R_PeEc,Medial_Temporal,Ventral_Multimodal,27,-9,-35,0.06,0.06


 
wm


,Brain Region,Network,x-MNI,y-MNI,z-MNI,Mean Coeff,σ Coeff
Glasser ROI,,,,,,,
R_EC,Medial_Temporal,Default,20,-11,-27,0.31,0.06
R_p32,Anterior_Cingulate_and_Medial_Prefrontal,Default,8,49,-3,0.24,0.19
L_PreS,Medial_Temporal,Default,-20,-31,-11,0.19,0.13
R_7m,Posterior_Cingulate,Default,5,-60,33,0.19,0.07
R_STSda,Auditory_Association,Default,51,-1,-17,0.11,0.09
...,...,...,...,...,...,...,...
L_AAIC,Insular_and_Frontal_Opercular,Orbito-Affective,-36,14,-11,-0.11,0.08
R_PeEc,Medial_Temporal,Ventral_Multimodal,27,-9,-35,0.18,0.08
L_TF,Medial_Temporal,Ventral_Multimodal,-42,-21,-31,0.05,0.10


In [19]:
avertask_abs = pd.read_csv(path+'mean_TASKS_brain_elnet_coeffs.csv', index_col=0)['0']
display(avertask_abs.sort_values(ascending=False))

R_6v_ROI      0.085657
R_PSL_ROI     0.084468
R_8BL_ROI     0.081000
R_PCV_ROI     0.073815
L_8BM_ROI     0.064677
                ...   
L_A5_ROI      0.009938
R_FOP4_ROI    0.009904
L_6a_ROI      0.009352
R_10d_ROI     0.009349
L_3a_ROI      0.008989
Name: 0, Length: 379, dtype: float64

In [25]:
#averaged tasks
#top20

avertask_abs = pd.read_csv(path+'mean_TASKS_brain_elnet_coeffs.csv', index_col=0)['0']#.sort_values(ascending=False)[:10]

tab1 = avertask_abs.abs().sort_values(ascending=False)[:20]
tab2 = avertask_abs.reindex(index = tab1.index)
#tab3 = 

tabnet = match_net_tab[match_net_tab['name_Glasser'].isin(list(tab1.index))].iloc[:,3:]
tabnet = tabnet.set_index(tabnet['name_Glasser'])
tabnet = tabnet.reindex(index=tab1.index)
tabnet['coeff'] = np.array(tab2)
tabnet = tabnet.reset_index()
tabnet = tabnet.drop('name_Glasser', axis=1)
tabnet = tabnet.rename(columns={'index': 'name_Glasser'})
tabnet['Cole_net_name'] = tabnet['Cole_net_name'].fillna('subcortex')
tabnet['freq'] = [tabnet['Cole_net_name'].value_counts()[val] for val in tabnet['Cole_net_name']]
tabnet = tabnet.sort_values(by=['freq', 'Cole_net_name', 'coeff'], ascending=False)
tabnet = tabnet.drop('freq', axis=1)
tabnet = tabnet.reset_index(drop=True)

init_lbl_gla = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]

tabnet['name_Glasser'] = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]
tabnet = tabnet.set_index(tabnet['name_Glasser'])
gla = gla_mni[gla_mni['regionName'].isin(tabnet['name_Glasser'])].loc[:,('regionName', 'regionLongName', 'Lobe', 'cortex', 'x-cog', 'y-cog', 'z-cog')]
gla = gla.set_index(gla['regionName'])
gla = gla.reindex(index = tabnet.index)

tabnet = pd.concat([tabnet, gla], axis=1)

col_nm = list(tabnet.columns)
col_nm = [col_nm[i] for i in [0,6, 1, 7,8,9, 2]]

tabnet =tabnet.loc[:,col_nm]
tabnet = tabnet.reset_index(drop=True)
tabnet['name_Glasser']=init_lbl_gla
nnms = ['Glasser ROI', 'Cortex', 'Network', 'x-MNI', 'y-MNI', 'z-MNI', 'Magnitude']
tabnet.columns = nnms
tabnet = tabnet.round(2)

tabnet['x-MNI'] = tabnet['x-MNI'].round().astype('Int64')
tabnet['y-MNI'] = tabnet['y-MNI'].round().astype('Int64')
tabnet['z-MNI'] = tabnet['z-MNI'].round().astype('Int64')
tabnet.index = tabnet['Glasser ROI']
tabnet.pop('Glasser ROI')
#tabnet = tabnet.dropna()

display(tabnet)
tabnet.to_csv(path+'taskAbsMag_top20_feature_coef.csv')
#tabnet.to_csv('/home/narunlab/Desktop/taskAbsMag_all_feature_coef.csv')

,Cortex,Network,x-MNI,y-MNI,z-MNI,Magnitude
Glasser ROI,,,,,,
R_8BL,Dorsolateral_Prefrontal,Default,11,43,48,0.08
R_STSda,Auditory_Association,Default,51,-1,-17,0.06
L_9m,Anterior_Cingulate_and_Medial_Prefrontal,Default,-7,54,22,0.06
R_47m,Orbital_and_Polar_Frontal,Default,32,31,-18,0.05
R_p32,Anterior_Cingulate_and_Medial_Prefrontal,Default,8,49,-3,0.05
L_8BM,Anterior_Cingulate_and_Medial_Prefrontal,Frontoparietal,-6,33,44,0.06
L_IFSa,Inferior_Frontal,Frontoparietal,-47,33,9,0.06
R_p10p,Orbital_and_Polar_Frontal,Frontoparietal,23,61,1,0.05
R_IFJp,Inferior_Frontal,Frontoparietal,36,7,28,0.05


In [27]:
#averaged tasks
#all parcels

avertask_abs = pd.read_csv(path+'mean_TASKS_brain_elnet_coeffs.csv', index_col=0)['0']#.sort_values(ascending=False)[:10]

tab1 = avertask_abs.abs().sort_values(ascending=False)#[:20]
tab2 = avertask_abs.reindex(index = tab1.index)
#tab3 = 

tabnet = match_net_tab[match_net_tab['name_Glasser'].isin(list(tab1.index))].iloc[:,3:]
tabnet = tabnet.set_index(tabnet['name_Glasser'])
tabnet = tabnet.reindex(index=tab1.index)
tabnet['coeff'] = np.array(tab2)
tabnet = tabnet.reset_index()
tabnet = tabnet.drop('name_Glasser', axis=1)
tabnet = tabnet.rename(columns={'index': 'name_Glasser'})
tabnet['Cole_net_name'] = tabnet['Cole_net_name'].fillna('subcortex')
tabnet['freq'] = [tabnet['Cole_net_name'].value_counts()[val] for val in tabnet['Cole_net_name']]
tabnet = tabnet.sort_values(by=['freq', 'Cole_net_name', 'coeff'], ascending=False)
tabnet = tabnet.drop('freq', axis=1)
tabnet = tabnet.reset_index(drop=True)

init_lbl_gla = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]

tabnet['name_Glasser'] = [(str(j[0])+str('_')+str(j[1])) for j in [i.split('_') for i in tabnet['name_Glasser']]]
tabnet = tabnet.set_index(tabnet['name_Glasser'])
gla = gla_mni[gla_mni['regionName'].isin(tabnet['name_Glasser'])].loc[:,('regionName', 'regionLongName', 'Lobe', 'cortex', 'x-cog', 'y-cog', 'z-cog')]
gla = gla.set_index(gla['regionName'])
gla = gla.reindex(index = tabnet.index)

tabnet = pd.concat([tabnet, gla], axis=1)

col_nm = list(tabnet.columns)
col_nm = [col_nm[i] for i in [0,6, 1, 7,8,9, 2]]

tabnet =tabnet.loc[:,col_nm]
tabnet = tabnet.reset_index(drop=True)
tabnet['name_Glasser']=init_lbl_gla
nnms = ['Glasser ROI', 'Cortex', 'Network', 'x-MNI', 'y-MNI', 'z-MNI', 'Magnitude']
tabnet.columns = nnms
tabnet = tabnet.round(2)

tabnet['x-MNI'] = tabnet['x-MNI'].round().astype('Int64')
tabnet['y-MNI'] = tabnet['y-MNI'].round().astype('Int64')
tabnet['z-MNI'] = tabnet['z-MNI'].round().astype('Int64')
tabnet.index = tabnet['Glasser ROI']
tabnet.pop('Glasser ROI')
#tabnet = tabnet.dropna()

display(tabnet)
#tabnet.to_csv(path+'taskAbsMag_top20_feature_coef.csv')
tabnet.to_csv(path+'taskAbsMag_all_feature_coef.csv')

,Cortex,Network,x-MNI,y-MNI,z-MNI,Magnitude
Glasser ROI,,,,,,
R_8BL,Dorsolateral_Prefrontal,Default,11,43,48,0.08
R_STSda,Auditory_Association,Default,51,-1,-17,0.06
L_9m,Anterior_Cingulate_and_Medial_Prefrontal,Default,-7,54,22,0.06
R_47m,Orbital_and_Polar_Frontal,Default,32,31,-18,0.05
R_p32,Anterior_Cingulate_and_Medial_Prefrontal,Default,8,49,-3,0.05
...,...,...,...,...,...,...
L_Pir,Insular_and_Frontal_Opercular,Orbito-Affective,-33,5,-18,0.02
R_PeEc,Medial_Temporal,Ventral_Multimodal,27,-9,-35,0.03
L_PeEc,Medial_Temporal,Ventral_Multimodal,-30,-9,-34,0.02


In [32]:
rest75_tabl=pd.read_csv(path+'rest_enet_coeffs_final_rescaled_mean.csv', index_col=0, header=None)[1]

In [57]:
#for resting state


#read tables and sort by absolute value
tab1 = pd.DataFrame(rest75_tabl.reindex(index=(rest75_tabl.abs().sort_values(ascending=False)[:20].index)))#[:20]  #top20 features
tab1.columns = ['Magnitude']
tab1.index.name = 'Glasser ROI'

#read index and split pairs to 2 indexes with purpose to find a Net name for each
ind1 = [i.split('_&_')[0] for i in  tab1.index]  #.replace("_ROI", "")
ind2 = [i.split('_&_')[1] for i in  tab1.index]

#insert new columns
tab1.insert(0, "ROI1", ind1)
tab1.insert(1, "ROI2", ind2)

match_net_tab.index = match_net_tab['name_Glasser']

#find match with match table

net1 = []
net2 = []
for index in tab1.index:
    if tab1.loc[index, 'ROI1'] not in match_net_tab['Cole_net_name']:
        net1+=['subcortex']
    else:
        net1+= [ match_net_tab.loc[ tab1.loc[index, 'ROI1'] , 'Cole_net_name'] ]
    if tab1.loc[index, 'ROI2'] not in match_net_tab['Cole_net_name']:
        net2+=['subcortex']
    else:
        net2+= [ match_net_tab.loc[ tab1.loc[index, 'ROI2'] , 'Cole_net_name'] ]

tab1.insert(2, "Network1", net1)  
tab1.insert(3, "Network2", net2) 


display(tab1)

tab1.to_csv(path+'restFCnet_top20_feature_coef.csv')


,ROI1,ROI2,Network1,Network2,Magnitude
Glasser ROI,,,,,
R_SCEF_ROI_&_L_IFJa_ROI,R_SCEF_ROI,L_IFJa_ROI,Cingulo-Opercular,Language,-6.439294
R_IP0_ROI_&_L_IFJp_ROI,R_IP0_ROI,L_IFJp_ROI,Dorsal_Attention,Frontoparietal,-4.607426
R_6ma_ROI_&_R_VMV3_ROI,R_6ma_ROI,R_VMV3_ROI,Cingulo-Opercular,Visual2,-4.218847
R_47l_ROI_&_L_a24pr_ROI,R_47l_ROI,L_a24pr_ROI,Default,Cingulo-Opercular,3.996803
R_FOP3_ROI_&_R_TPOJ1_ROI,R_FOP3_ROI,R_TPOJ1_ROI,Cingulo-Opercular,Language,3.885781
R_7PC_ROI_&_R_MBelt_ROI,R_7PC_ROI,R_MBelt_ROI,Somatomotor,Auditory,-3.781697
L_POS1_ROI_&_L_MBelt_ROI,L_POS1_ROI,L_MBelt_ROI,Default,Auditory,-3.774758
R_6v_ROI_&_L_STV_ROI,R_6v_ROI,L_STV_ROI,Somatomotor,Language,3.774758
L_Pir_ROI_&_L_PHT_ROI,L_Pir_ROI,L_PHT_ROI,Orbito-Affective,Dorsal_Attention,3.552714


In [58]:
stst = pd.Series(pd.get_dummies(list(zip(tab1['Network1'].values, tab1['Network2'].values))).sum().sort_values(ascending=False))
stst.index = ['_'.join(sorted(list(i))) for i in stst.index]
cindex = sorted(set(stst.index))
stst_new = {}
for i in cindex:
    stst_new[i] = stst[i].sum()
display(pd.Series(stst_new).sort_values(ascending=False))

Cingulo-Opercular_Frontoparietal          2
Cingulo-Opercular_Language                2
Cingulo-Opercular_Visual2                 2
Dorsal_Attention_Frontoparietal           2
Auditory_Cingulo-Opercular                1
Auditory_Default                          1
Auditory_Somatomotor                      1
Cingulo-Opercular_Default                 1
Cingulo-Opercular_Orbito-Affective        1
Cingulo-Opercular_Posterior_Multimodal    1
Default_Language                          1
Dorsal_Attention_Orbito-Affective         1
Language_Somatomotor                      1
Language_Visual2                          1
Orbito-Affective_Visual2                  1
Somatomotor_Visual2                       1
dtype: int64

In [61]:
#for resting state

#all features

#read tables and sort by absolute value
tab1 = pd.DataFrame(rest75_tabl.reindex(index=(rest75_tabl.abs().sort_values(ascending=False).index)))
tab1.columns = ['Magnitude']
tab1.index.name = 'Glasser ROI'

#read index and split pairs to 2 indexes with purpose to find a Net name for each
ind1 = [i.split('_&_')[0] for i in  tab1.index]  #.replace("_ROI", "")
ind2 = [i.split('_&_')[1] for i in  tab1.index]

#insert new columns
tab1.insert(0, "ROI1", ind1)
tab1.insert(1, "ROI2", ind2)

match_net_tab.index = match_net_tab['name_Glasser']

#find match with match table

net1 = []
net2 = []
for index in tab1.index:
    if tab1.loc[index, 'ROI1'] not in match_net_tab['Cole_net_name']:
        net1+=['subcortex']
    else:
        net1+= [ match_net_tab.loc[ tab1.loc[index, 'ROI1'] , 'Cole_net_name'] ]
    if tab1.loc[index, 'ROI2'] not in match_net_tab['Cole_net_name']:
        net2+=['subcortex']
    else:
        net2+= [ match_net_tab.loc[ tab1.loc[index, 'ROI2'] , 'Cole_net_name'] ]

tab1.insert(2, "Network1", net1)  
tab1.insert(3, "Network2", net2) 


display(tab1)
tab1.to_csv(path+'restFCnet_all_feature_coef.csv')


,ROI1,ROI2,Network1,Network2,Magnitude
Glasser ROI,,,,,
R_SCEF_ROI_&_L_IFJa_ROI,R_SCEF_ROI,L_IFJa_ROI,Cingulo-Opercular,Language,-6.439294
R_IP0_ROI_&_L_IFJp_ROI,R_IP0_ROI,L_IFJp_ROI,Dorsal_Attention,Frontoparietal,-4.607426
R_6ma_ROI_&_R_VMV3_ROI,R_6ma_ROI,R_VMV3_ROI,Cingulo-Opercular,Visual2,-4.218847
R_47l_ROI_&_L_a24pr_ROI,R_47l_ROI,L_a24pr_ROI,Default,Cingulo-Opercular,3.996803
R_FOP3_ROI_&_R_TPOJ1_ROI,R_FOP3_ROI,R_TPOJ1_ROI,Cingulo-Opercular,Language,3.885781
...,...,...,...,...,...
R_9-46d_ROI_&_L_PI_ROI,R_9-46d_ROI,L_PI_ROI,Cingulo-Opercular,Cingulo-Opercular,0.000000
L_a24pr_ROI_&_L_IFJp_ROI,L_a24pr_ROI,L_IFJp_ROI,Cingulo-Opercular,Frontoparietal,0.000000
R_5m_ROI_&_R_5L_ROI,R_5m_ROI,R_5L_ROI,Somatomotor,Somatomotor,0.000000


In [62]:
stst = pd.Series(pd.get_dummies(list(zip(tab1['Network1'].iloc[:100].values, tab1['Network2'].iloc[:100].values))).sum().sort_values(ascending=False))
stst.index = ['_'.join(sorted(list(i))) for i in stst.index]
cindex = sorted(set(stst.index))
stst_new = {}
for i in cindex:
    stst_new[i] = stst[i].sum()
display(pd.Series(stst_new).sort_values(ascending=False))

Default_Visual2                           10
Cingulo-Opercular_Default                  8
Cingulo-Opercular_Visual2                  7
Somatomotor_Visual2                        5
Default_Language                           5
Dorsal_Attention_Frontoparietal            4
Auditory_Somatomotor                       4
Cingulo-Opercular_Orbito-Affective         3
Language_Visual2                           3
Default_Default                            3
Cingulo-Opercular_Dorsal_Attention         3
Cingulo-Opercular_Language                 2
Default_Frontoparietal                     2
Dorsal_Attention_Somatomotor               2
Cingulo-Opercular_Cingulo-Opercular        2
Auditory_Cingulo-Opercular                 2
Default_Visual1                            2
Frontoparietal_Visual2                     2
Cingulo-Opercular_Frontoparietal           2
Cingulo-Opercular_subcortex                2
Posterior_Multimodal_Somatomotor           2
Cingulo-Opercular_Somatomotor              2
Posterior_